In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import pickle

2025-10-11 08:44:34.000168: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-11 08:44:34.087579: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%reload_ext autoreload
%autoreload 2

from spin_system import SpinSystem
from spin_system import interactions, plots

In [26]:
def langevin_step(system, beta, lr):
    epsilon = tf.sqrt(2 * lr / beta)
    with tf.GradientTape() as tape:
        E = system.compute_pairwise_energies()

    grad = (epsilon**2) * tape.gradient(E, system.spin_state)
    noise = tf.random.normal(system.spin_state.shape) * epsilon
    new_spins = system.spin_state - lr * grad + noise
    system.spin_state.assign(new_spins)
    return E

def optimizer(system, beta, lr, n_steps, record_interval=100):
    n_records = n_steps // record_interval

    evolution = tf.TensorArray(
        dtype=tf.float32, size=n_records, dynamic_size=False, clear_after_read=False
    )

    for step in tf.range(n_steps):
        langevin_step(system, beta, lr)

        if (step + 1) % record_interval == 0:
            idx = step // record_interval
            evolution = evolution.write(idx, tf.identity(system.spin_state))

    return evolution.stack()


In [31]:
lattice_dim = 2
n_steps = 3000
lattice_length = 32
lattice_replicas = 64
# betas = tf.sort(
#     tf.concat(
#         [
#             tf.linspace(0.01, 0.9, 10), 
#             tf.linspace(2.1, 3, 10),
#             tf.linspace(0.91, 2.0, 20)
#         ], 
#         axis = 0
#     )
# )

interaction_matrix = interactions.gaussian_interaction(lattice_dim, lattice_length)
spin_system = SpinSystem(
    lattice_dim=lattice_dim,
    lattice_length=lattice_length,
    lattice_replicas=lattice_replicas,
    interaction_matrix=interaction_matrix,
    model="spherical",
    initial_magnetization=0.0,
)

In [32]:
result = optimizer(spin_system, 1, 1e-3, n_steps)

In [33]:
result

<tf.Tensor: shape=(30, 64, 32, 32), dtype=float32, numpy=
array([[[[ 1.16252017e+00,  1.67275429e+00, -2.44184732e+00, ...,
          -6.93476677e-01, -8.51072967e-01,  6.15337491e-01],
         [ 1.23719490e+00, -1.09061706e+00,  1.08430612e+00, ...,
          -1.15285444e+00, -5.74043870e-01,  2.68680871e-01],
         [-9.96793389e-01,  2.29800034e+00,  5.83692193e-01, ...,
          -3.53781551e-01, -2.63629627e+00,  2.68922389e-01],
         ...,
         [-4.32766914e-01, -1.14894927e+00,  4.99712855e-01, ...,
          -1.10482156e+00, -1.58645630e+00, -4.10833299e-01],
         [-4.69663858e-01, -1.27076018e+00,  9.11875665e-01, ...,
           2.76301432e+00,  7.30889440e-02, -1.08558619e-02],
         [-1.00548543e-01,  1.54060471e+00, -6.91091001e-01, ...,
           2.48241812e-01,  1.76301503e+00, -3.05478454e-01]],

        [[ 1.01268578e+00,  1.27651834e+00, -2.75901866e+00, ...,
          -4.31546092e-01, -5.69498479e-01, -1.18347697e-01],
         [ 1.46538961e+00, -6.

In [41]:
plots.plot_spin_evolution(result[:, -10, :, :])